In [ ]:
"""
Prepare Davison et al. 2023 steady state to use in summer paper

"""

In [ ]:
import xarray as xr
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
home_path = '/bettik/burgardc/'

In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)
    
plot_path='/bettik/burgardc/PLOTS/summer_paper_plots/'

In [ ]:
inputpath_davison = home_path+'/DATA/SUMMER_PAPER/raw/'
file_davison_pd = pd.read_csv(inputpath_davison + 'Data_Davison_SteadyState.csv',delimiter=';',header=0,index_col=0).dropna(how='all',axis=1)

In [ ]:
file_davison = file_davison_pd.to_xarray()

In [ ]:
file_davison.isel(ice_shelf=0)

In [ ]:
row_list = []
for kisf in tqdm(file_isf.Nisf):
    nname = file_isf['isf_name'].sel(Nisf=kisf).values
    
    if nname == 'Filchner-Ronne':
        abs_values = file_davison[['discharge_obs','basal_melt_obs','calving_obs','SMB_obs']].sel(ice_shelf='Filchner').drop('ice_shelf') + file_davison[['discharge_obs','basal_melt_obs','calving_obs','SMB_obs']].sel(ice_shelf='Ronne').drop('ice_shelf')
        unc_values = np.sqrt(file_davison[['discharge_unc','basal_melt_unc','calving_unc','SMB_unc']].sel(ice_shelf='Filchner').drop('ice_shelf')**2 
                            + file_davison[['discharge_unc','basal_melt_unc','calving_unc','SMB_unc']].sel(ice_shelf='Ronne').drop('ice_shelf')**2)
        davison_row = xr.merge([abs_values,unc_values])
    elif nname == 'Ross':
        abs_values = file_davison[['discharge_obs','basal_melt_obs','calving_obs','SMB_obs']].sel(ice_shelf='Ross_East').drop('ice_shelf') + file_davison[['discharge_obs','basal_melt_obs','calving_obs','SMB_obs']].sel(ice_shelf='Ross_West').drop('ice_shelf')
        unc_values = np.sqrt(file_davison[['discharge_unc','basal_melt_unc','calving_unc','SMB_unc']].sel(ice_shelf='Ross_East').drop('ice_shelf')**2 
                             + file_davison[['discharge_unc','basal_melt_unc','calving_unc','SMB_unc']].sel(ice_shelf='Ross_West').drop('ice_shelf')**2)
        davison_row = xr.merge([abs_values,unc_values])
        
    elif nname == 'Vincennes Bay':
        davison_row  = file_davison.sel(ice_shelf='Vincennes_Bay').drop('ice_shelf')
    elif nname == 'George VI':
        davison_row  = file_davison.sel(ice_shelf='George_VI').drop('ice_shelf')
    elif nname == 'Larsen B':
        davison_row  = file_davison.sel(ice_shelf='Larsen_B').drop('ice_shelf')
    elif nname == 'Larsen C':
        davison_row  = file_davison.sel(ice_shelf='Larsen_C').drop('ice_shelf')
    elif nname == 'Larsen D':
        davison_row  = file_davison.sel(ice_shelf='Larsen_D').drop('ice_shelf')
    elif nname == 'Larsen E':
        davison_row  = file_davison.sel(ice_shelf='Larsen_E').drop('ice_shelf')
    elif nname == 'Larsen F':
        davison_row  = file_davison.sel(ice_shelf='Larsen_F').drop('ice_shelf')
    elif nname == 'Larsen G':
        davison_row  = file_davison.sel(ice_shelf='Larsen_G').drop('ice_shelf')
    elif nname == 'Prince Harald':
        davison_row  = file_davison.sel(ice_shelf='Prince_Harald').drop('ice_shelf')
    elif nname == 'Moscow Univ.':
        davison_row  = file_davison.sel(ice_shelf='Moscow_University').drop('ice_shelf')
    elif nname == 'Riiser-Larsen':
        davison_row  = file_davison.sel(ice_shelf='Riiser_Larsen').drop('ice_shelf')
    elif nname == 'Conger/Glenzer':
        davison_row  = file_davison.sel(ice_shelf='Conger_Glenzer').drop('ice_shelf')
    elif nname == 'Roi Baudouin':
        davison_row  = file_davison.sel(ice_shelf='Baudouin').drop('ice_shelf')
    elif nname == 'Wilma/Robert/Downer':
        davison_row  = file_davison.sel(ice_shelf='Wilma_Robert_Downer').drop('ice_shelf')
    elif nname == 'Tracy Tremenchus':
        davison_row  = file_davison.sel(ice_shelf='Tracy_Tremenchus').drop('ice_shelf')
    elif nname == 'Rayner/Thyer':
        davison_row  = file_davison.sel(ice_shelf='Rayner_Thyer').drop('ice_shelf')
    elif nname == 'Pine Island':
        davison_row  = file_davison.sel(ice_shelf='Pine_Island').drop('ice_shelf')
    elif nname == 'Stancomb Brunt':
        davison_row  = file_davison.sel(ice_shelf='Brunt_Stancomb').drop('ice_shelf')
    elif nname == 'Edward VIII':
        davison_row  = file_davison.sel(ice_shelf='Edward_VIII').drop('ice_shelf')
    else:
        davison_row  = file_davison.sel(ice_shelf=nname).drop('ice_shelf')
    
    row_list.append(davison_row.assign_coords({'Nisf': kisf}))
    
new_ds = xr.concat(row_list, dim='Nisf')

In [ ]:
new_ds.to_netcdf(inputpath_davison + 'steadystate_davison23.nc')

In [ ]:
test_allvv

In [ ]:
test_allvv_list = []
for vv in ['discharge','SMB','calving','melt']:
    test_pd_abs = pd.read_csv(inputpath_davison + 'Davison23_'+vv+'_abs.csv',delimiter=';',header=0,index_col=0).dropna(how='all',axis=1)
    test_pd_unc = pd.read_csv(inputpath_davison + 'Davison23_'+vv+'_unc.csv',delimiter=';',header=0,index_col=0).dropna(how='all',axis=1)

    test_list = []
    for ccol in test_pd.columns:
        test_float_abs = test_pd_abs[ccol].str.replace('[A-Za-z]', '').str.replace(',', '.').astype(float)
        test_float_unc = test_pd_unc[ccol].str.replace('[A-Za-z]', '').str.replace(',', '.').astype(float)

        test_list.append(xr.merge([test_float_abs.to_xarray().rename(vv+'_abs').assign_coords({'time': int(ccol)}),
                                  test_float_unc.to_xarray().rename(vv+'_unc').assign_coords({'time': int(ccol)})]))
    test_allyy = xr.concat(test_list, dim='time')
    test_allvv_list.append(test_allyy)
test_allvv = xr.merge(test_allvv_list)    
test_allvv = test_allvv.rename({'Ice shelf':'ice_shelf'})

In [ ]:
test_allvv.rename({'Ice shelf':'ice_shelf'})

In [ ]:
row_list = []
for kisf in tqdm(file_isf.Nisf):
    nname = file_isf['isf_name'].sel(Nisf=kisf).values
    
    if nname == 'Filchner-Ronne':
        abs_values = test_allvv[['discharge_abs','melt_abs','calving_abs','SMB_abs']].sel(ice_shelf='Filchner').drop('ice_shelf') + test_allvv[['discharge_abs','melt_abs','calving_abs','SMB_abs']].sel(ice_shelf='Ronne').drop('ice_shelf')
        unc_values = np.sqrt(test_allvv[['discharge_unc','melt_unc','calving_unc','SMB_unc']].sel(ice_shelf='Filchner').drop('ice_shelf')**2 
                            + test_allvv[['discharge_unc','melt_unc','calving_unc','SMB_unc']].sel(ice_shelf='Ronne').drop('ice_shelf')**2)
        davison_row = xr.merge([abs_values,unc_values])
    elif nname == 'Ross':
        abs_values = test_allvv[['discharge_abs','melt_abs','calving_abs','SMB_abs']].sel(ice_shelf='Ross_East').drop('ice_shelf') + test_allvv[['discharge_abs','melt_abs','calving_abs','SMB_abs']].sel(ice_shelf='Ross_West').drop('ice_shelf')
        unc_values = np.sqrt(test_allvv[['discharge_unc','melt_unc','calving_unc','SMB_unc']].sel(ice_shelf='Ross_East').drop('ice_shelf')**2 
                             + test_allvv[['discharge_unc','melt_unc','calving_unc','SMB_unc']].sel(ice_shelf='Ross_West').drop('ice_shelf')**2)
        davison_row = xr.merge([abs_values,unc_values])
        
    elif nname == 'Vincennes Bay':
        davison_row  = test_allvv.sel(ice_shelf='Vincennes_Bay').drop('ice_shelf')
    elif nname == 'George VI':
        davison_row  = test_allvv.sel(ice_shelf='George_VI').drop('ice_shelf')
    elif nname == 'Larsen B':
        davison_row  = test_allvv.sel(ice_shelf='Larsen_B').drop('ice_shelf')
    elif nname == 'Larsen C':
        davison_row  = test_allvv.sel(ice_shelf='Larsen_C').drop('ice_shelf')
    elif nname == 'Larsen D':
        davison_row  = test_allvv.sel(ice_shelf='Larsen_D').drop('ice_shelf')
    elif nname == 'Larsen E':
        davison_row  = test_allvv.sel(ice_shelf='Larsen_E').drop('ice_shelf')
    elif nname == 'Larsen F':
        davison_row  = test_allvv.sel(ice_shelf='Larsen_F').drop('ice_shelf')
    elif nname == 'Larsen G':
        davison_row  = test_allvv.sel(ice_shelf='Larsen_G').drop('ice_shelf')
    elif nname == 'Prince Harald':
        davison_row  = test_allvv.sel(ice_shelf='Prince_Harald').drop('ice_shelf')
    elif nname == 'Moscow Univ.':
        davison_row  = test_allvv.sel(ice_shelf='Moscow_University').drop('ice_shelf')
    elif nname == 'Riiser-Larsen':
        davison_row  = test_allvv.sel(ice_shelf='Riiser_Larsen').drop('ice_shelf')
    elif nname == 'Conger/Glenzer':
        davison_row  = test_allvv.sel(ice_shelf='Conger_Glenzer').drop('ice_shelf')
    elif nname == 'Roi Baudouin':
        davison_row  = test_allvv.sel(ice_shelf='Baudouin').drop('ice_shelf')
    elif nname == 'Wilma/Robert/Downer':
        davison_row  = test_allvv.sel(ice_shelf='Wilma_Robert_Downer').drop('ice_shelf')
    elif nname == 'Tracy Tremenchus':
        davison_row  = test_allvv.sel(ice_shelf='Tracy_Tremenchus').drop('ice_shelf')
    elif nname == 'Rayner/Thyer':
        davison_row  = test_allvv.sel(ice_shelf='Rayner_Thyer').drop('ice_shelf')
    elif nname == 'Pine Island':
        davison_row  = test_allvv.sel(ice_shelf='Pine_Island').drop('ice_shelf')
    elif nname == 'Stancomb Brunt':
        davison_row  = test_allvv.sel(ice_shelf='Brunt_Stancomb').drop('ice_shelf')
    elif nname == 'Edward VIII':
        davison_row  = test_allvv.sel(ice_shelf='Edward_VIII').drop('ice_shelf')
    else:
        davison_row  = test_allvv.sel(ice_shelf=nname).drop('ice_shelf')
    
    row_list.append(davison_row.assign_coords({'Nisf': kisf}))
    
new_ds = xr.concat(row_list, dim='Nisf')

In [ ]:
new_ds.to_netcdf(inputpath_davison + 'varying_conditions_davison23.nc')

CHANGING AREA FOR EVERY YEAR

In [ ]:
file_greene_pd = pd.read_csv(inputpath_davison + 'Greene22_area_abs.csv',delimiter=';',header=0,index_col=1).dropna(how='all',axis=1)

In [ ]:
test_allyy

In [ ]:
file_greene_pd = pd.read_csv(inputpath_davison + 'Greene22_area_abs.csv',delimiter=';',header=0,index_col=1).dropna(how='all',axis=1)    

test_list = []
for ccol in file_greene_pd.columns[1::]:
    #print(ccol)
    test_float_abs = file_greene_pd[ccol].str.replace('[A-Za-z]', '').str.replace(',', '.').astype(float)

    test_list.append(test_float_abs.to_xarray().rename('area_abs').assign_coords({'time': int(ccol)}))
test_allyy = xr.concat(test_list, dim='time').rename({'Ice Shelf':'ice_shelf'})

In [ ]:
row_list = []
for kisf in tqdm(file_isf.Nisf):
    nname = file_isf['isf_name'].sel(Nisf=kisf).values
    
    if nname == 'Filchner-Ronne':
        davison_row = test_allyy.sel(ice_shelf='Filchner').drop('ice_shelf') + test_allyy.sel(ice_shelf='Ronne').drop('ice_shelf')

    elif nname == 'Ross':
        davison_row = test_allyy.sel(ice_shelf='Ross East').drop('ice_shelf') + test_allyy.sel(ice_shelf='Ross West').drop('ice_shelf')

        
    elif nname == 'Moscow Univ.':
        davison_row  = test_allyy.sel(ice_shelf='Moscow University').drop('ice_shelf')
    elif nname == 'Riiser-Larsen':
        davison_row  = test_allyy.sel(ice_shelf='Riiser_Larsen').drop('ice_shelf')
    elif nname == 'Conger/Glenzer':
        davison_row  = test_allyy.sel(ice_shelf='Conger Glenzer').drop('ice_shelf')
    elif nname == 'Roi Baudouin':
        davison_row  = test_allyy.sel(ice_shelf='Baudouin').drop('ice_shelf')
    elif nname == 'Wilma/Robert/Downer':
        davison_row  = test_allyy.sel(ice_shelf='Wilma_Robert_Downer').drop('ice_shelf')
    elif nname == 'Tracy Tremenchus':
        davison_row  = test_allyy.sel(ice_shelf='Tracy_Tremenchus').drop('ice_shelf')
    elif nname == 'Rayner/Thyer':
        davison_row  = test_allyy.sel(ice_shelf='Rayner_Thyer').drop('ice_shelf')

    elif nname == 'Stancomb Brunt':
        davison_row  = test_allyy.sel(ice_shelf='Brunt_Stancomb').drop('ice_shelf')
    elif nname == 'Edward VIII':
        davison_row  = test_allyy.sel(ice_shelf='Edward_VIII').drop('ice_shelf')
    else:
        davison_row  = test_allyy.sel(ice_shelf=nname).drop('ice_shelf')
    
    row_list.append(davison_row.assign_coords({'Nisf': kisf}))
    
new_ds = xr.concat(row_list, dim='Nisf')

In [ ]:
test_allyy.ice_shelf